In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-12-24


Found rundate 2023-12-23 10:00:02 < 2023-12-24 at commit fa8d0dbc; returning commit 089f5e6


'2023-12-24 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-12-24 10:00:02, NJSP was reporting 592 YTD deaths
Current YTD Deaths (2024-12-24 10:00:04-05:00): 676
Previous year YTD Deaths (adjusted; 2023-12-24 10:00:02): 592.034356445122
Projected 2024 total: 691.9052911462763
14.2% change


FAUQStats cache miss: 2023, Sun Dec 24 10:00:02 EST 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,STREET,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,NaN,Mulberry St,1.0,0.0,0.0,1.0,0.0,Mulberry St,0.0,2023-01-01 01:15:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,322,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,NaN,0.0,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,NaN,South Randolphville Road,1.0,1.0,0.0,0.0,0.0,South Randolphville Road,0.0,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,NaN,Outwater Ln,1.0,0.0,0.0,1.0,0.0,Outwater Ln,0.0,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,NaN,Washington St,1.0,0.0,0.0,1.0,0.0,Washington St,NaN,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12893,11,Mercer,1112,Robbinsville Twsp,95,New Jersey Turnpike MP 59.9,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2023-12-22 04:28:00-05:00
12892,14,Morris,1439,Wharton Boro,15,State Highway 15 MP 2.4,1.0,0.0,1.0,0.0,0.0,NaN,NaN,2023-12-22 05:42:00-05:00
12894,12,Middlesex,1219,Sayreville Boro,35,35,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2023-12-22 06:47:00-05:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,31,2,18,4,54
Bergen,15,5,19,2,39
Burlington,31,4,8,1,44
Camden,18,3,15,0,36
Cape May,8,0,0,0,8
Cumberland,19,6,7,0,27
Essex,17,10,27,0,48
Gloucester,18,8,6,0,29
Hudson,9,2,7,1,18


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

97.9% through the year, 2.1% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,17,18,31
Bergen,driver,21,20,15
Burlington,driver,24,26,31
Camden,driver,19,18,18
Cape May,driver,4,4,8
...,...,...,...,...
Salem,crashes,11,11,13
Somerset,crashes,20,21,19
Sussex,crashes,6,6,13


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,57,8,33,2,18
Bergen,40,2,14,5,22
Burlington,50,2,34,6,8
Camden,37,0,17,4,16
Cape May,9,0,8,0,0
Cumberland,27,0,19,6,7
Essex,52,0,19,12,28
Gloucester,31,0,19,9,7
Hudson,18,2,8,2,7


In [9]:
projected.to_csv(PROJECTED_CSV)